In [157]:
def dict_init(team_year_dict,team,type_prefixes,stat_keys):
    team_year_dict[team]={}
    for prefix in type_prefixes:
        for key in stat_keys:
            team_year_dict[team][prefix+key]=0

In [187]:
import os
import glob
import yaml

match_types=['Test','T20','ODI']
type_prefixes=['t_','t2_','o_']
stat_keys=['wins','losses','other','sixes','fours','other_runs','bowler_wicket','fielder_wicket','keeper_wicket','sixes_given','fours_given','other_runs_given','bowler_wicket_given','fielder_wicket_given','keeper_wicket_given']
countries=['West Indies','Australia','Sri Lanka','India','New Zealand','South Africa','England','Pakistan','Bangladesh','Afghanistan','Zimbabwe']

all_years={}

# create all_years which is dict of form: all_year{'year':{'team_name':{stats}}}

for dir_name in os.listdir('data/yearwise_stats'):
    
    if dir_name=='.DS_Store':
        continue
    year=dir_name[:4]
    print(year)
    yaml_paths=glob.glob('data/yearwise_stats/'+dir_name+'/*.yaml')
    
    team_year_dict={}
    
    for path in yaml_paths:
        try :
            with open(path,'r') as file:
                    # The FullLoader parameter handles the conversion from YAML
                    # scalar values to Python the dictionary format


                    match = yaml.load(file, Loader=yaml.FullLoader)

                    match_type=match['info']['match_type']
                    if match_type in match_types :

                        prefix=type_prefixes[match_types.index(match_type)]

                        teams=match['info']['teams']

#                         if not all( team in countries for team in teams):
#                             print(teams)
#                             continue
                        for team in teams:
                            if team not in team_year_dict.keys():
                                dict_init(team_year_dict,team,type_prefixes,stat_keys)

                            if 'winner' in match['info']['outcome'].keys():
                                if team == match['info']['outcome']['winner']:
                                    team_year_dict[team][prefix+'wins']+=1
                                else:
                                    team_year_dict[team][prefix+'losses']+=1
                            else:
                                team_year_dict[team][prefix+'other']+=1

                        for innings in match['innings']:
                            team=innings[list(innings.keys())[0]]['team']
                            other_team=[t for t in teams if t!=team][0]

                            deliveries=innings[list(innings.keys())[0]]['deliveries']
                            for delivery in deliveries:
                                delivery_key=list(delivery.keys())[0]
                                runs=delivery[delivery_key]['runs']
                                if runs['batsman']==4:
                                    team_year_dict[team][prefix+'fours']+=1
                                    team_year_dict[other_team][prefix+'fours_given']+=1

                                elif runs['batsman']==6:
                                    team_year_dict[team][prefix+'sixes']+=1
                                    team_year_dict[other_team][prefix+'sixes_given']+=1

                                else:
                                    team_year_dict[team][prefix+'other_runs']+=runs['batsman']
                                    team_year_dict[other_team][prefix+'other_runs_given']+=runs['batsman']

                                team_year_dict[team][prefix+'other_runs']+=runs['extras']
                                team_year_dict[other_team][prefix+'other_runs_given']+=runs['extras']




                                if 'wicket' in delivery[delivery_key].keys():
                                    wicket=delivery[delivery_key]['wicket']
    #                                 print(wicket)
    #                                 print(path)
                                    if isinstance(wicket,list):
                                        wicket=wicket[0]
                                    if wicket['kind']=='bowled' or wicket['kind']=='lbw':
                                        team_year_dict[other_team][prefix+'bowler_wicket']+=1
                                        team_year_dict[team][prefix+'bowler_wicket_given']+=1
                                    if wicket['kind']=='caught' or wicket['kind']=='run out':
                                        team_year_dict[other_team][prefix+'fielder_wicket']+=1
                                        team_year_dict[team][prefix+'fielder_wicket_given']+=1
                                    if wicket['kind']=='stumped':
                                        team_year_dict[other_team][prefix+'keeper_wicket']+=1
                                        team_year_dict[team][prefix+'keeper_wicket_given']+=1

#         print(len(team_year_dict.keys()))
        except Exception as e:
            print(path)
            print(e)
        
    for team,stats in team_year_dict.items():
        for prefix in type_prefixes:
            num_matches=stats[prefix+'wins']+stats[prefix+'losses']+stats[prefix+'other']
            print(num_matches)
            if num_matches!=0:
                for key in stat_keys[3:]:
                    stats[prefix+key]/=num_matches
                        
        all_years[year]=team_year_dict


2015
0
10
0
0
10
0
0
14
0
0
14
0
0
10
0
0
7
0
14
5
26
13
1
19
9
4
23
0
11
29
0
9
14
0
13
0
0
5
13
8
4
32
11
4
25
10
5
15
8
10
27
0
5
9
0
7
1
0
6
0
8
9
24
0
14
0
5
5
18
0
16
0
0
17
0
0
5
0
0
3
0
0
14
0
0
16
0
0
6
0
0
10
0
0
5
0
0
4
7
0
8
0
0
8
0
2014
0
14
0
0
15
0
8
12
25
10
7
24
0
11
0
0
11
0
0
8
0
0
14
0
7
15
13
9
10
16
9
13
18
0
10
0
8
10
19
0
5
5
4
3
15
9
7
16
7
10
18
11
9
32
0
3
4
0
0
2
0
0
6
0
16
0
0
7
2
0
14
0
0
16
0
0
3
10
0
10
0
0
9
0
0
17
0
0
14
0
0
8
0
0
4
0
0
4
1
0
7
0
0
0
2
2009
12
8
18
3
3
16
14
9
22
6
12
18
0
0
2
0
2
3
13
9
38
0
0
2
0
0
15
8
12
21
6
10
31
0
5
2
9
11
20
11
13
25
0
16
0
0
13
0
0
16
0
0
14
0
0
13
0
0
2
0
0
15
0
0
13
0
0
14
0
0
0
1
2013
0
16
0
0
16
0
14
7
21
14
6
23
0
18
0
0
17
0
0
18
0
9
8
28
7
5
23
0
9
0
0
10
1
4
9
30
6
6
13
6
4
9
8
1
34
8
12
34
0
6
2
12
7
19
0
6
0
0
6
0
0
6
3
0
4
0
0
4
0
0
19
0
0
16
0
0
16
0
0
16
0
0
6
0
0
5
0
0
2
5
0
4
3
0
3
0
0
2
0
2012
10
11
33
10
17
15
9
14
17
15
14
15
10
15
13
10
13
17
11
13
25
0
5
4
0
5
1
0
16
0
0
17
0
0
18
0
6
16
18

In [190]:

with open('data/clean_data/all_years.yml', 'w') as outfile:
    yaml.dump(all_years, outfile, default_flow_style=False)

In [196]:
import csv
with open('data/clean_data/all_years.csv', 'w', newline="") as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(['Year','Country']+[prefix+x for prefix in type_prefixes for x in stat_keys])
    for year,team_dict in all_years.items():
        
        for name, stats in team_dict.items():
            if name not in countries:
                continue
            csv_row=[year,name]+[stats[prefix+x] for prefix in type_prefixes for x in stat_keys]
#             for prefix in type_prefixes: 
#                 csv_row+=[stats[prefix+x] for x in stat_keys]
            writer.writerow(csv_row)

In [211]:
from datetime import datetime

t20_rows=[]

yaml_paths=glob.glob('data/t20s_male/*.yaml')

team_year_dict={}
yaml_paths.sort()

for path in yaml_paths:
    with open(path,'r') as file:
        match = yaml.load(file, Loader=yaml.FullLoader)
        match_type=match['info']['match_type']
        if match_type == 'T20' :

#             prefix=type_prefixes[match_types.index(match_type)]

            teams=match['info']['teams']

            if not all( team in countries for team in teams):
#                 print(teams)
                continue
    
            match_date=""
            if isinstance(match['info']['dates'][0],str):
                print("str:",match['info']['dates'][0])
                match_date=[match['info']['dates'][0]]
                
            else:
                match_date=[match['info']['dates'][0].strftime("%Y-%m-%d")]
                print("not str:",match_date)
                
            row=match_date+teams


            if 'winner' in match['info']['outcome'].keys():
                if teams[0]==match['info']['outcome']['winner']:
                    row+=[1,0]
                else:
                    row+=[0,1]
            else:
                row+=[0,0]


            t20_rows.append(row)


not str: ['2017-02-17']
not str: ['2017-02-19']
not str: ['2017-02-22']
not str: ['2016-06-18']
not str: ['2016-06-20']
not str: ['2016-06-22']
not str: ['2017-01-03']
not str: ['2017-01-06']
not str: ['2017-01-08']
not str: ['2017-02-17']
not str: ['2017-06-21']
not str: ['2017-06-23']
not str: ['2017-06-25']
not str: ['2017-01-26']
not str: ['2017-01-29']
not str: ['2017-02-01']
not str: ['2016-08-27']
not str: ['2016-08-28']
not str: ['2016-09-23']
not str: ['2016-09-24']
not str: ['2016-09-27']
not str: ['2017-03-26']
not str: ['2017-03-30']
not str: ['2017-04-04']
not str: ['2017-04-06']
not str: ['2017-04-01']
not str: ['2017-04-02']
not str: ['2017-06-02']
not str: ['2017-06-03']
not str: ['2017-06-05']
not str: ['2017-07-09']
str: 2019-02-01
str: 2019-02-03
str: 2019-03-19
str: 2019-03-22
str: 2019-03-24
str: 2019-05-05
str: 2019-03-05
str: 2019-03-08
str: 2019-03-10
str: 2019-02-27
str: 2019-10-27
str: 2019-10-30
str: 2019-11-01
str: 2019-11-03
str: 2019-11-05
str: 2019-11-08


not str: ['2013-11-21']
not str: ['2014-03-16']
not str: ['2014-03-21']
not str: ['2014-03-22']
str: 2014-03-22
not str: ['2014-03-23']
not str: ['2014-03-23']
not str: ['2014-03-24']
not str: ['2014-03-25']
not str: ['2014-03-27']
not str: ['2014-03-28']
not str: ['2014-03-28']
not str: ['2014-03-29']
not str: ['2014-03-30']
not str: ['2014-03-30']
not str: ['2014-03-31']
not str: ['2014-04-01']
not str: ['2014-04-01']
not str: ['2014-04-03']
not str: ['2014-04-04']
not str: ['2014-04-06']
not str: ['2013-11-20']
not str: ['2013-11-22']
not str: ['2014-02-12']
not str: ['2014-02-14']
not str: ['2015-01-09']
not str: ['2015-01-11']
not str: ['2014-10-05']
not str: ['2014-07-05']
not str: ['2014-07-06']
not str: ['2014-08-27']
not str: ['2015-01-14']
not str: ['2014-12-04']
not str: ['2015-06-23']
not str: ['2015-08-31']
not str: ['2014-12-05']
not str: ['2014-11-05']
not str: ['2014-11-07']
not str: ['2014-11-09']
not str: ['2016-02-19']
not str: ['2016-02-21']
not str: ['2015-07-05']


In [214]:
import csv
with open('data/clean_data/t20_results.csv', 'w', newline="") as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(['Date','Team1','Team2','Result1','Result2'])
    for row in t20_rows:
        writer.writerow(row)
                    

In [229]:
t20_rows.sort(key= lambda row: row[0])
t20_rows[:10]
    

[['2005-02-17', 'New Zealand', 'Australia', 0, 1],
 ['2005-06-13', 'England', 'Australia', 1, 0],
 ['2005-10-21', 'South Africa', 'New Zealand', 0, 1],
 ['2006-01-09', 'Australia', 'South Africa', 1, 0],
 ['2006-02-16', 'New Zealand', 'West Indies', 0, 0],
 ['2006-02-24', 'South Africa', 'Australia', 1, 0],
 ['2006-06-15', 'England', 'Sri Lanka', 0, 1],
 ['2006-08-28', 'England', 'Pakistan', 0, 1],
 ['2006-12-01', 'South Africa', 'India', 0, 1],
 ['2006-12-22', 'New Zealand', 'Sri Lanka', 0, 1]]

In [239]:

len_batch=5
count=0
win_rate_rows=[]
for year in range(2008,2018):
    for country in countries:
        
        print(country)
        batch_matches=[row for row in t20_rows if (row[1]==country or row[2]==country) and int(row[0][:4])>=year][:len_batch]
        win_rate=0
        for row in batch_matches:
            if row[1]==country:
                win_rate+=row[3]
            else:
                win_rate+=row[4]
        win_rate/=len_batch
        #         win_rate=sum()
        win_rate_rows.append([year,country,win_rate])
#         if len([row for row in t20_rows if (row[1]==country or row[2]==country) and int(row[0][:4])==year]) >=5:
#             count+=1
#         print(len([row for row in t20_rows if row[1]==country and int(row[0][:4])==year]))
count



West Indies
Australia
Sri Lanka
India
New Zealand
South Africa
England
Pakistan
Bangladesh
Afghanistan
Zimbabwe
West Indies
Australia
Sri Lanka
India
New Zealand
South Africa
England
Pakistan
Bangladesh
Afghanistan
Zimbabwe
West Indies
Australia
Sri Lanka
India
New Zealand
South Africa
England
Pakistan
Bangladesh
Afghanistan
Zimbabwe
West Indies
Australia
Sri Lanka
India
New Zealand
South Africa
England
Pakistan
Bangladesh
Afghanistan
Zimbabwe
West Indies
Australia
Sri Lanka
India
New Zealand
South Africa
England
Pakistan
Bangladesh
Afghanistan
Zimbabwe
West Indies
Australia
Sri Lanka
India
New Zealand
South Africa
England
Pakistan
Bangladesh
Afghanistan
Zimbabwe
West Indies
Australia
Sri Lanka
India
New Zealand
South Africa
England
Pakistan
Bangladesh
Afghanistan
Zimbabwe
West Indies
Australia
Sri Lanka
India
New Zealand
South Africa
England
Pakistan
Bangladesh
Afghanistan
Zimbabwe
West Indies
Australia
Sri Lanka
India
New Zealand
South Africa
England
Pakistan
Bangladesh
Afghanistan
Z

0

In [248]:
win_rate_rows[:15]

[[2008, 'West Indies', 0.4],
 [2008, 'Australia', 0.8],
 [2008, 'Sri Lanka', 0.6],
 [2008, 'India', 0.4],
 [2008, 'New Zealand', 0.2],
 [2008, 'South Africa', 0.6],
 [2008, 'England', 0.8],
 [2008, 'Pakistan', 0.6],
 [2008, 'Bangladesh', 0.0],
 [2008, 'Afghanistan', 0.0],
 [2008, 'Zimbabwe', 0.2],
 [2009, 'West Indies', 0.6],
 [2009, 'Australia', 0.6],
 [2009, 'Sri Lanka', 0.8],
 [2009, 'India', 0.4]]

In [259]:
import csv
with open('data/clean_data/win_rates.csv', 'w', newline="") as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(['Year','Country','Win-Rate'])
    for row in win_rate_rows:
        writer.writerow(row)

In [260]:
import csv
with open('data/clean_data/stats_with_winrates.csv', 'w', newline="") as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(['Year','Country']+[prefix+x for prefix in type_prefixes for x in stat_keys]+['Win-Rate'])
    for row in win_rate_rows:
        if row[1] in all_years[str(row[0]-1)].keys() :
            csv_row=[str(row[0]-1),row[1]]+[all_years[str(row[0]-1)][row[1]][prefix+x] for prefix in type_prefixes for x in stat_keys]+[row[2]]
            writer.writerow(csv_row)
     

In [256]:
count=0
for row in win_rate_rows:
    print(all_years[str(row[0]-1)])
    if row[1] in all_years[str(row[0]-1)].keys() :
        all_years[str(row[0]-1)][row[1]]
        count+=1
count

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



107

In [147]:
countries=['West Indies','Australia','Sri Lanka','Zimbabwe','India','New Zealand','South Africa','England','Pakistan','Bangladesh']

In [233]:
all_years['2008']['India']

{'t_wins': 6,
 't_losses': 4,
 't_other': 5,
 't_sixes': 3.933333333333333,
 't_fours': 66.8,
 't_other_runs': 326.93333333333334,
 't_bowler_wicket': 5.8,
 't_fielder_wicket': 8.066666666666666,
 't_keeper_wicket': 0.4,
 't_sixes_given': 1.8,
 't_fours_given': 59.93333333333333,
 't_other_runs_given': 320.3333333333333,
 't_bowler_wicket_given': 6.533333333333333,
 't_fielder_wicket_given': 9.133333333333333,
 't_keeper_wicket_given': 0.4,
 't2_wins': 0,
 't2_losses': 1,
 't2_other': 0,
 't2_sixes': 0.0,
 't2_fours': 3.0,
 't2_other_runs': 62.0,
 't2_bowler_wicket': 0.0,
 't2_fielder_wicket': 1.0,
 't2_keeper_wicket': 0.0,
 't2_sixes_given': 2.0,
 't2_fours_given': 4.0,
 't2_other_runs_given': 47.0,
 't2_bowler_wicket_given': 2.0,
 't2_fielder_wicket_given': 8.0,
 't2_keeper_wicket_given': 0.0,
 'o_wins': 19,
 'o_losses': 7,
 'o_other': 2,
 'o_sixes': 3.7142857142857144,
 'o_fours': 21.642857142857142,
 'o_other_runs': 133.57142857142858,
 'o_bowler_wicket': 1.8214285714285714,
 'o_fi